In [1]:
import re
from pathlib import Path

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 1000)  # 省略されないように表示件数を設定

In [3]:
DATA_DIR = Path('../data/')
files_path = list(DATA_DIR.joinpath('timetable_2023').glob('*.xlsx'))

In [4]:
dfs = {}
pattern = re.compile('[A-Z]')
for file_path in files_path:
    df = pd.read_excel(file_path, sheet_name=1)  # 0番目(X科)は時間割の神エクセル、1番目(Sheet1)がシラバス形式
    df['授業コード'] = df['授業コード'].astype(str).apply(lambda x: x.zfill(4))  # 授業コードは左ゼロ埋めで4桁
    df['ｺｰｽ'] = df['ｺｰｽ'].astype(str).apply(lambda x: x.strip())  # \u3000(全角スペース)が含まれているので除去
    df['学年'] = df['学年'].astype(str)  # 数値と文字列が混在しているので文字列に統一
    key = pattern.findall(str(file_path))[0]
    dfs[key] = df
df = dfs['M']
cols = df.columns

In [5]:
# 備考: 講義名前の「（特）」「（留）」は特別授業と留学生対象科目、講義名後に「（19生以前のみ）」などの特記事項あり
# 備考: ◎印-必修科目、□印-選択必修科目、○印-選択科目、△印-自由科目、■印(選択必修科目)-履修制限科目（同群の再履修者専用科目）
# 備考: ②-1・②-2等は自動的に学籍番号で分割され、1組・2組等はレベル別(履修登録前に発表される掲示で確認)
df.head()

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
0,0001,スタディスキル,◎,1,M・F・E・T,1,④-1,高橋明,NaN
1,0002,スタディスキル,◎,1,M・F・E・T,1,④-2,福山,NaN
2,0003,スタディスキル,◎,1,M・F・E・T,1,④-3,福山,NaN
3,0004,スタディスキル,◎,1,M・F・E・T,1,④-4,小林,NaN
4,0029,（特）スタディスキル,◎,1,M・F・E・T,2,NaN,小田切,NaN


# 授業コード

In [6]:
# 単純な疑問: 授業コード0002と0003に関して、教員が同じため同じ講義と思われるのですが、授業コードが異なるのはなぜでしょうか？
df[:4]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
0,0001,スタディスキル,◎,1,M・F・E・T,1,④-1,高橋明,NaN
1,0002,スタディスキル,◎,1,M・F・E・T,1,④-2,福山,NaN
2,0003,スタディスキル,◎,1,M・F・E・T,1,④-3,福山,NaN
3,0004,スタディスキル,◎,1,M・F・E・T,1,④-4,小林,NaN


# 必選
一応xcatが組む際には使わないと想定されるので、各々の意味はよくわからないが無視できる。

In [7]:
# ○: 選択科目であり、◎: 必修科目であるという矛盾が生じているように思えます。
# □: 選択必修科目との違いは何でしょうか？
df[503:507]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
503,0505,英会話Ⅰ,○◎,1,M・F・E・T,1・2,NaN,大木,NaN
504,0523,英会話Ⅰ,○◎,1,M・F・E・T,1・2,NaN,大木,NaN
505,0520,英会話Ⅱ,○◎,1,M・F・E・T,1・2,NaN,大木,NaN
506,0525,英会話Ⅱ,○◎,1,M・F・E・T,1・2,NaN,大木,NaN


In [8]:
# また、■は選択必修科目かつ再履修者専用という位置づけでしょうか？
df[df['必選'] == '■']

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
407,8200,基礎力学Ⅰ-a,■,1,M・F,1,NaN,佐藤航,NaN
412,8204,基礎力学Ⅱ-a,■,1,M・F,2,NaN,南葉,NaN


In [9]:
# しかし、再履修前の科目(つまり同名の科目)は存在しません...
target_names = df[df['必選'] == '■']['授業科目'].tolist()
df[df['授業科目'].isin(target_names)]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
407,8200,基礎力学Ⅰ-a,■,1,M・F,1,NaN,佐藤航,NaN
412,8204,基礎力学Ⅱ-a,■,1,M・F,2,NaN,南葉,NaN


# コース

In [10]:
# コースがない場合(nan)はどういった扱いになるのでしょうか？
df['ｺｰｽ'].unique()

array(['M・F・E・T', 'M・F・E', 'T', 'M・E', 'F', 'M・F・T', 'E', 'J', 'F・T',
       'M・E・T', 'S', 'J・F・T', 'M', 'S・F', 'J・T', 'M・T', 'M・F', 'S・J・F・T',
       'S・J', 'nan'], dtype=object)

# 学年

In [11]:
# 「1・2」と「1･2」が混在しているので処理の時には注意！
# TODO: 表記ゆれの除去
df['学年'].unique()

array(['1', '2', '1・2', '3', '1･2', '2・3', '4', '3・4'], dtype=object)

# クラス

In [12]:
# クラスがない場合(nan)はどういった扱いになるのでしょうか？
df['ｸﾗｽ'].unique()

array(['④-1', '④-2', '④-3', '④-4', nan, '②-1', '②-2', '1組', '2組', '3組',
       '③-1', '③-2', '③-3'], dtype=object)

# 教員名

In [13]:
# ちょっと厄介なので処理の時に注意
df['教員名'].unique()

array(['高橋明', '福山', '小林', '小田切', '三浦', '中山', '石田', '上野', '楠',
       '＊三浦・師玉真・佐藤史・多田・中畑・山田陽・山本崇',
       '〔＊三浦〕\u3000師玉真・山本聡・佐藤史・小田切・山田陽・山本崇・中畑', 'ﾑﾙｱｶ', '〔＊師玉〕', '師玉真',
       '＊師玉真・小田切', '＊伊藤勝・冨澤', '室井', '山本崇', '多田', '山本聡', '渡辺', '荒船', '佐々木',
       '竹村', '趙', '中畑', '於保', '義澤', '山崎', '佐藤史', '鍋倉', '兼頭', '田辺', '比嘉',
       '泉', 'ﾊｽｹﾞﾚﾙ', '三橋', '山田博', '山田陽', '高坂', '〔＊高嶋〕\u3000勝浦・松本剛',
       '〔＊高嶋〕\u3000岩崎', '〔＊高嶋〕\u3000松本彰', '〔＊高嶋〕\u3000戸松', '浅川', '町田',
       '宮城', '山本', '大木', '岩本', '大平', '天谷', '河野', '照井', '木村茂', '〔＊大木〕',
       '新谷', 'Ｄ', '松下', '牛膓', '田口', '池川', '菊地', '小机', '神谷亮', '南葉', '兒玉',
       '熊谷', '格和', '＊納富・高取・杉村・三枝亮・宮崎・村田・臼杵・鈴木聡・塩野・須藤・前田', '＊金井・広井・若杉・中尾',
       '＊金井・瑞慶覧・前田', '〔＊斎藤〕', '師玉礼', '江連', '吉岡', '高橋一', '渡部', '＊照井・中根',
       '今井', '萩野', '木村', '林', '佐藤智', '＊水野・丸山・小池', '川島', '吉川',
       '＊今井・熊谷・長尾・髙村・酒井・高取', '有川', '＊今井・小机・渡部・萩野',
       '＊川島・佐藤智・中根・林・渡部・熊谷・山中・Ａ', '＊川島・高石・木村茂・林・吉岡・吉川・根本・山中・大場・Ａ',
       '＊照井・渡邉・稲谷・杉本・三桝・佐藤昌', '＊木村茂・渡邉・稲谷・杉本・三桝・佐藤昌', '大場', '＊林・M科全教員',


In [14]:
# 以下の講義は担当教員がいない(nan)のですが、そのような講義は存在するのでしょうか？
df[df['教員名'].isnull()]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
352,1096,機械工学プロジェクトⅡ,◎,2,M・E,1,②-1,NaN,NaN
355,1097,機械工学プロジェクトⅡ,◎,2,M・E,1,②-2,NaN,NaN
507,1113,Academic English for Global Leader Ⅰ,○,1,F,1,NaN,NaN,NaN
508,1115,Academic English for Global Leader Ⅲ,○,1,F,2,NaN,NaN,NaN
509,1114,Academic English for Global Leader Ⅱ,○,1,F,1,NaN,NaN,NaN
510,1116,Academic English for Global Leader Ⅳ,○,1,F,2,NaN,NaN,NaN


In [15]:
# 以下のような教員名の場合、講義の各回ごとに担当教員が異なるという認識でよろしいのでしょうか？
df[19:21]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
19,0070,現代社会講座,◎,1,M・F・E,1,②-1,＊三浦・師玉真・佐藤史・多田・中畑・山田陽・山本崇,NaN
20,0071,現代社会講座,◎,1,M・F・E,1,②-2,＊三浦・師玉真・佐藤史・多田・中畑・山田陽・山本崇,NaN


In [16]:
# 以下の講義は「代表者であるが授業は担当しない」表記ですが、どういう状態なのでしょうか？
exists_idx = df['教員名'].dropna().index
df_sub = df.iloc[exists_idx]

df_sub.loc[df_sub['教員名'].str.match('〔＊.*〕') & (df_sub['教員名'].apply(len) <= 5)]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
23,0088,新聞理解表現演習Ⅰ,◎,1,T,1,NaN,〔＊師玉〕,NaN
24,0089,新聞理解表現演習Ⅱ,◎,1,T,1,NaN,〔＊師玉〕,NaN
180,0544,総合英語演習,○,1,M・F・E・T,1,NaN,〔＊大木〕,NaN
181,0545,総合英語演習,○,1,M・F・E・T,1,NaN,〔＊大木〕,NaN
182,0546,総合英語演習,○,1,M・F・E・T,1・2,NaN,〔＊大木〕,NaN
183,0547,総合英語演習,○,1,M・F・E・T,1・2,NaN,〔＊大木〕,NaN
238,0850,早期インターンシップ準備演習,◎,1,T,1,NaN,〔＊斎藤〕,NaN
239,0853,早期インターンシップ,◎,1,T,2,NaN,〔＊斎藤〕,NaN


# 最大人数

In [17]:
df['最大人数'].unique()

array([ nan, 180.,  50.])

In [18]:
# 最大人数がない場合は、時間割を決める際にどのように部屋を割り当てているのでしょうか。
# 前年度の教室の規模をそのまま流用している形でしょうか？
df[df['最大人数'].isnull()].head()

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
0,0001,スタディスキル,◎,1,M・F・E・T,1,④-1,高橋明,NaN
1,0002,スタディスキル,◎,1,M・F・E・T,1,④-2,福山,NaN
2,0003,スタディスキル,◎,1,M・F・E・T,1,④-3,福山,NaN
3,0004,スタディスキル,◎,1,M・F・E・T,1,④-4,小林,NaN
4,0029,（特）スタディスキル,◎,1,M・F・E・T,2,NaN,小田切,NaN


# その他の疑問

In [19]:
# 別の疑問: -cと-dは別授業なのでしょうか？8205と8207の違いがわかりません。
# 別の疑問: 基礎力学Ⅱを再履修になったEコースの学生はどうなるのでしょうか？
df.loc[df['授業科目'].str.contains('基礎力学Ⅱ')]

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
412,8204,基礎力学Ⅱ-a,■,1,M・F,2,NaN,南葉,NaN
413,8205,基礎力学Ⅱ-c,□,2,M・F・E,1,1組,山本,NaN
414,8206,基礎力学Ⅱ-c,□,2,M・E,1,2組,門田,NaN
415,8207,基礎力学Ⅱ-d,□,2,M・F・E,1,NaN,栗田,NaN


In [20]:
df

,授業コード,授業科目,必選,コマ数,ｺｰｽ,学年,ｸﾗｽ,教員名,最大人数
0,0001,スタディスキル,◎,1,M・F・E・T,1,④-1,高橋明,NaN
1,0002,スタディスキル,◎,1,M・F・E・T,1,④-2,福山,NaN
2,0003,スタディスキル,◎,1,M・F・E・T,1,④-3,福山,NaN
3,0004,スタディスキル,◎,1,M・F・E・T,1,④-4,小林,NaN
4,0029,（特）スタディスキル,◎,1,M・F・E・T,2,NaN,小田切,NaN
5,0030,（特）スタディスキル,◎,1,M・F・E・T,1・2,NaN,小田切,NaN
6,0034,情報社会と情報倫理,○,1,M・F・E,3,NaN,三浦,NaN
7,0034,情報社会と情報倫理,□,1,T,3,NaN,三浦,NaN
8,0035,技術者倫理,○,1,M・E,3,NaN,中山,NaN
9,0035,技術者倫理,◎,1,F,3,NaN,中山,NaN
